<a href="https://colab.research.google.com/github/mohannashahrad/Borealis_AI_Plant_Tree_Project/blob/main/preprocessing_scripts/GDP_DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Import all the required libraries
import pandas as pd
import requests
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [49]:
# Define all the functions

def load_DF(url, skip):
  data = StringIO(requests.get(url).text)
  return pd.read_csv(data, skiprows = skip)

In [50]:
# Load GDP Dataset
gdp_df = load_DF('https://raw.githubusercontent.com/mohannashahrad/Borealis_AI_Plant_Tree_Project/main/Data/GDP_Data_1960_2020.csv', range(0, 3))
gdp_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.054634e+08,4.876025e+08,5.964236e+08,6.953044e+08,7.648871e+08,8.721387e+08,9.584632e+08,1.082980e+09,1.245688e+09,1.320475e+09,1.379961e+09,1.531944e+09,1.665101e+09,1.722799e+09,1.873453e+09,1.920112e+09,1.941341e+09,2.021229e+09,2.228492e+09,2.330726e+09,2.424581e+09,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.727850e+09,2.790849e+09,2.962905e+09,2.983637e+09,3.092430e+09,3.202189e+09,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,1.929193e+10,1.970186e+10,2.147035e+10,2.570500e+10,2.350165e+10,2.678117e+10,2.912019e+10,3.014009e+10,3.284101e+10,3.770296e+10,4.027158e+10,4.442789e+10,4.824852e+10,6.291444e+10,7.816508e+10,8.334408e+10,8.311919e+10,9.488479e+10,1.062295e+11,1.243617e+11,1.563411e+11,1.602022e+11,1.544998e+11,1.595823e+11,1.458618e+11,1.302968e+11,1.468640e+11,1.795424e+11,1.887960e+11,1.943300e+11,2.121032e+11,2.205216e+11,2.199773e+11,2.203721e+11,2.245112e+11,2.524746e+11,2.518900e+11,2.648551e+11,2.497234e+11,2.464216e+11,2.676138e+11,2.414727e+11,2.470095e+11,3.258902e+11,4.047997e+11,4.705097e+11,5.321390e+11,6.115618e+11,6.662911e+11,6.692327e+11,8.036886e+11,8.962561e+11,9.131974e+11,9.276555e+11,9.563187e+11,8.930997e+11,8.547519e+11,9.622690e+11,9.840320e+11,9.778092e+11,8.984741e+11,NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,1.400000e+09,1.673333e+09,1.373333e+09,1.408889e+09,1.748887e+09,1.831109e+09,1.595555e+09,1.733333e+09,2.155555e+09,2.366667e+09,2.555556e+09,2.953333e+09,3.300000e+09,3.697940e+09,3.641723e+09,3.478788e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.055180e+09,4.515559e+09,5.226779e+09,6.209138e+09,6.971286e+09,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,1.980707e+10,NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040732e+10,1.113130e+10,1.194684e+10,1.268022e+10,1.384262e+10,1.486682e+10,1.583747e+10,1.443065e+10,1.488470e+10,1.688703e+10,2.351148e+10,2.083891e+10,2.527234e+10,3.128296e+10,4.422741e+10,5.145977e+10,6.214756e+10,6.533410e+10,7.122053e+10,8.865431e+10,1.120641e+11,2.110652e+11,1.872184e+11,1.381556e+11,1.142961e+11,1.165413e+11,1.075290e+11,1.103540e+11,1.089753e+11,1.017985e+11,1.218377e+11,1.174914e+11,1.183168e+11,9.718677e+10,8.569306e+10,1.074030e+11,1.190436e+11,1.199833e+11,1.226213e+11,1.301987e+11,1.341502e+11,1.418625e+11,1.705319e+11,1.973842e+11,2.458565e+11,3.021108e+11,3.843363e+11,4.518661e+11,5.530312e+11,4.925458e+11,5.802173e+11,6.584282e+11,7.169352e+11,8.078189e+11,8.469431e+11,7.574921e+11,6.874847e+11,6.809891e+11,7.381313e+11,7.920789e+11,7.865850e+11,NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.934074e+09,5.553824e+09,5.553824e+09,5.787824e+09,6.135166e+09,7.558613e+09,7.076794e+09,8.089279e+09,8.775116e+09,1.020792e+10,1.123628e+10,NaN,NaN,NaN,3.390500e+09,5.561222e+09,7.526964e+09,7.649716e+09,6.506619e+09,6.152937e+09,9.129635e+09,8.936064e+09,1.528559e+10,1.781271e+10,2.355205e+10,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.941719e+10,6.230691e+10,NaN


In [51]:
# Load Pop Dataset
population_df = load_DF('https://raw.githubusercontent.com/mohannashahrad/Borealis_AI_Plant_Tree_Project/main/Data/Population_Data_1950_2100.csv', -1)
population_df.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486


In [52]:
gdp_df.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 65'], axis=1, inplace=True)
gdp_df = pd.melt(gdp_df, id_vars='Country Name', var_name='Time', value_name='GDP (USD)')
gdp_df.rename(columns={"Country Name": "Country"}, inplace=True)
gdp_df['Time'] = gdp_df['Time'].astype(int)
gdp_df = gdp_df[gdp_df.Time >= 1990]
gdp_df = gdp_df[gdp_df.Time <= 2020]
display(gdp_df)

,Country,Time,GDP (USD)
7980,Aruba,1990,7.648871e+08
7981,Africa Eastern and Southern,1990,2.121032e+11
7982,Afghanistan,1990,NaN
7983,Africa Western and Central,1990,1.218377e+11
7984,Angola,1990,1.123628e+10
...,...,...,...
16221,Kosovo,2020,7.611402e+09
16222,"Yemen, Rep.",2020,NaN
16223,South Africa,2020,3.019236e+11
16224,Zambia,2020,1.932005e+10


In [53]:
population_df.drop(['LocID', 'VarID', 'Variant', 'MidPeriod', 'PopMale', 'PopFemale'], axis=1, inplace=True)
population_df.rename(columns={"Location": "Country"}, inplace=True)
population_df['Time'] = population_df['Time'].astype(int)
population_df = population_df[population_df.Time >= 1990]
population_df = population_df[population_df.Time <= 2020]
display(population_df)

,Country,Time,PopTotal,PopDensity
40,Afghanistan,1990,12412.311,19.012
41,Afghanistan,1991,13299.016,20.370
42,Afghanistan,1992,14485.543,22.188
43,Afghanistan,1993,15816.601,24.227
44,Afghanistan,1994,17075.728,26.155
...,...,...,...,...
280847,Zimbabwe,2020,14862.927,38.420
280864,Zimbabwe,2020,14862.927,38.420
280881,Zimbabwe,2020,14862.927,38.420
280898,Zimbabwe,2020,14862.927,38.420


In [54]:
gdp_index = gdp_df.set_index(['Country', 'Time'])
pop_index = population_df.set_index(['Country', 'Time'])
merged_df = gdp_index.join(pop_index, how='outer')
merged_df.head()


GDP (USD)   PopTotal  PopDensity
Country     Time                                  
Afghanistan 1990        NaN  12412.311      19.012
            1991        NaN  13299.016      20.370
            1992        NaN  14485.543      22.188
            1993        NaN  15816.601      24.227
            1994        NaN  17075.728      26.155

In [55]:
merged_df.reset_index(inplace=True)
merged_df.to_csv("pop_gdp.csv")